In [1]:
from scipy import stats
import arviz as az
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import seaborn as sns
import pandas as pd
from theano import shared
from sklearn import preprocessing
import pickle
import sklearn
from sklearn import preprocessing
from sklearn.datasets import make_regression
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,PolynomialFeatures
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.model_selection import train_test_split,KFold,RepeatedKFold,StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_percentage_error
#pd.set_option("display.max_rows", None, "display.max_columns", None)
print('Running on PyMC3 v{}'.format(pm.__version__))
import networkx as nx
import random
from scipy.stats import spearmanr
from numpy import mean,std
from random import choice, choices, shuffle, sample
from itertools import combinations, groupby
from IPython.display import display, HTML
!pip install table-evaluator
from table_evaluator import load_data, TableEvaluator
from sklearn.decomposition import PCA

Running on PyMC3 v3.11.2


In [3]:
a_file = open("scripts/data/Network_Metrics_real_dataset.pkl", "rb")
data = pickle.load(a_file)
data=data.drop(columns=['repo'])
df=data.iloc[:,[0,1,2,3,7,8,19]]
df=df.sort_values(by='no_of_nodes').reset_index(drop=True)
df

,no_of_nodes,no_of_edges,average_degree,average_clustering_coefficient,diameter,radius,powerlaw_exponent
0,25,85,6.800000,0.767825,3,2,6.580027
1,41,78,3.804878,0.738816,3,2,3.816859
2,42,123,5.857143,0.709830,3,2,5.054764
3,42,49,2.333333,0.250232,2,1,3.527568
4,43,143,6.651163,0.687416,4,2,4.182046
...,...,...,...,...,...,...,...
72,349,2133,12.223496,0.756432,5,3,2.310369
73,363,4191,23.090909,0.752535,4,3,2.142328
74,377,5522,29.294430,0.727573,6,3,5.255193
75,387,6626,34.242894,0.711831,4,3,2.212416


In [4]:
df=df.sort_values(by='average_degree').reset_index(drop=True)

# Data Synthesis

In [5]:
from scipy.stats import beta
#find parameters of beta distribution
a,b, loc,scale = stats.beta.fit(df.no_of_nodes)
# sample values from fitted distribution
node_samples = beta.rvs(a,b, loc,scale, size=100)

In [6]:
# log transformation
df=np.log10(df)

In [7]:
# bayesian model edges

edges_values = df['no_of_edges'].values.reshape(-1, 1)

with pm.Model() as regression_model_nodes_edges:
    
    nodes_values = pm.Data("nodes_values", df['no_of_nodes'].values.reshape(-1, 1))

    alpha = pm.Normal('alpha', mu = 15.13437422, sd =.5)
  
    beta = pm.Normal('beta', mu = -28.74787971, sd = .5)
    gamma = pm.Normal('gamma', mu = 22.3704513, sd = .5)
    delta = pm.Normal('delta', mu = -7.47195216, sd = .5)
    zeta = pm.Normal('zeta', mu = 0.94787594, sd = .5)

    epsilon = pm.HalfNormal('epsilon', sd = .01)
    
    by_mean = alpha + beta * nodes_values + gamma * nodes_values**2 + delta * nodes_values**3 + zeta * nodes_values**4
    
    Ylikelihood = pm.Normal('Ylikelihood', mu = by_mean, sd = epsilon, observed = edges_values)
        
    step = pm.NUTS()

    regression_trace_nodes_edges = pm.sample(1000, step, chains=1)

<ipython-input-7-b2c16ba2bdf9>:24: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  regression_trace_nodes_edges = pm.sample(1000, step, chains=1)
Sequential sampling (1 chains in 1 job)
NUTS: [epsilon, zeta, delta, gamma, beta, alpha]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 115 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [ ]:
# log transformation of sampled nodes 
node_samples_transformed=np.log10(node_samples)
with regression_model_nodes_edges:
    pm.set_data({"nodes_values": np.array(node_samples_transformed).reshape(-1, 1)})
    posterior_predictive_nodes_edges = pm.sample_posterior_predictive(regression_trace_nodes_edges)

In [ ]:
samples_nodes_edges=posterior_predictive_nodes_edges  

# select from different models
sample_no=random.choices(range(1000),k=100) 
samples_lst=[]
for x in range(100): 
  model_number = sample_no[x]
  model_value = samples_nodes_edges['Ylikelihood'][model_number][x]
  samples_lst.append(model_value[0])

full_prior_samples = pd.DataFrame(samples_lst, columns=['no_of_edges'])
full_prior_samples['no_of_nodes']=np.array(node_samples_transformed)
full_prior_samples=full_prior_samples.drop_duplicates().sort_values(by='no_of_nodes')

In [ ]:
df_synthetic=full_prior_samples[['no_of_nodes','no_of_edges']]
df_synthetic=10**df_synthetic
df_synthetic['average_degree']=(df_synthetic['no_of_edges']*2)/(df_synthetic['no_of_nodes'])
df_synthetic=np.log10(df_synthetic)

In [ ]:
# bayesian model CC

avg_clustering_values = df['average_clustering_coefficient'].values.reshape(-1, 1)

with pm.Model() as regression_model_degree_clustering:

    avg_degree_values = pm.Data("avg_degree_values", df['average_degree'].values.reshape(-1, 1))  

    alpha = pm.Normal('alpha', mu = -2.57778591, sd =.5)
    
    beta = pm.Normal('beta', mu = 8.67701697, sd = .5)
    gamma = pm.Normal('gamma', mu = -11.64685216, sd = .5)
    delta = pm.Normal('delta', mu = 6.97481184, sd = .5)
    zeta = pm.Normal('zeta', mu = -1.55689117, sd = .5)
    
    epsilon = pm.HalfNormal('epsilon', sd = 0.001)
    
    by_mean = alpha + beta * avg_degree_values + gamma * avg_degree_values**2 + delta * avg_degree_values**3 + zeta * avg_degree_values**4 
    
    Ylikelihood = pm.Normal('Ylikelihood', mu = by_mean, sd = epsilon, observed = avg_clustering_values)
    
    step = pm.NUTS()
    
    regression_trace_degree_clustering = pm.sample(1000, step, chains=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
Sequential sampling (1 chains in 1 job)
NUTS: [epsilon, zeta, delta, gamma, beta, alpha]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 122 seconds.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [ ]:
with regression_model_degree_clustering:
    pm.set_data({"avg_degree_values": df_synthetic['average_degree'].values.reshape(-1, 1)})
    posterior_predictive_degree_clustering = pm.sample_posterior_predictive(regression_trace_degree_clustering)

In [ ]:
samples_degree_clustering=posterior_predictive_degree_clustering

# select from different models
sample_no=random.choices(range(1000),k=100)
samples_lst=[]
for x in range(100):
  model_number = sample_no[x]
  model_value = samples_degree_clustering['Ylikelihood'][model_number][x]
  samples_lst.append(model_value[0])

full_prior_samples = pd.DataFrame(samples_lst, columns=['average_clustering_coefficient'])
full_prior_samples['average_degree']=df_synthetic.average_degree.values   
full_prior_samples=full_prior_samples.drop_duplicates().sort_values(by='average_degree')

In [ ]:
df_synthetic['average_clustering_coefficient']=full_prior_samples['average_clustering_coefficient']

In [ ]:
df_synthetic=10**df_synthetic
df_synthetic.no_of_nodes=df_synthetic.no_of_nodes.round(0)
df_synthetic.no_of_edges=df_synthetic.no_of_edges.round(0)
df_synthetic=df_synthetic.sort_values(by='no_of_nodes').reset_index(drop=True)


In [ ]:
df=data.iloc[:,[0,1,2,3,7,8,19]]
df=df.sort_values(by='average_degree').reset_index(drop=True)

In [ ]:
# powerlaw exponent synthesis
from scipy.stats import exponnorm
#fit
a, loc,scale = stats.exponnorm.fit(df.powerlaw_exponent)
# sample values
samples_powerlaw = exponnorm.rvs(a, loc,scale, size=100)  
df_synthetic['powerlaw_exponent'] = samples_powerlaw

In [ ]:
df_synthetic=df_synthetic.sort_values(by='no_of_nodes').reset_index(drop=True)
df=df.sort_values(by='no_of_nodes').reset_index(drop=True)

In [ ]:
# diameter synthesis
from collections import Counter
from scipy import stats
def get_distribution(dist):
  # choose distribution
  if dist==1:
    lowerBound=0;upperBound=30
  elif dist==2:
    lowerBound=30;upperBound=50
  elif dist==3:
    lowerBound=50;upperBound=77
  dias=df.diameter[lowerBound:upperBound]
  counts=dict(Counter(dias))
  # find counts
  probabs_abs=[]
  lst=list(range(1,9))
  for value in lst:
    try:
      probabs_abs.append(counts[value])
    except:
      probabs_abs.append(0)
  # find probabilites
  probabs=[float(i)/sum(probabs_abs) for i in probabs_abs]
  discrete_dist = stats.rv_discrete(name='custm', values=(lst, probabs))
  
  return discrete_dist

In [ ]:
# get length of each groups
sample1_len = len(df_synthetic[df_synthetic.no_of_nodes<=df.no_of_nodes[30]])
sample2_len = len(df_synthetic[(df_synthetic.no_of_nodes>df.no_of_nodes[30]) & (df_synthetic.no_of_nodes<=df.no_of_nodes[50])])
sample3_len = len(df_synthetic[df_synthetic.no_of_nodes>df.no_of_nodes[50]])

In [ ]:
samples_dia1=get_distribution(1).rvs(size=sample1_len)
samples_dia2=get_distribution(2).rvs(size=sample2_len)
samples_dia3=get_distribution(3).rvs(size=sample3_len)

In [ ]:
dias_lst=list(samples_dia1)
dias_lst.extend(list(samples_dia2));dias_lst.extend(list(samples_dia3))
#dias_lst
df_synthetic['diameter']=dias_lst

In [ ]:
# getting radius
import math
df_synthetic['radius']=df_synthetic['diameter'].apply(lambda x: math.ceil(x/2))
rand_choice=random.choices([0,1], weights = [0.961,0.039], k = 100)        # weights based on given data 
df_synthetic['radius']=df_synthetic['radius']+rand_choice

In [25]:
df_synthetic=df_synthetic.sort_values(by='no_of_nodes').reset_index(drop=True)
df_synthetic=df_synthetic[['no_of_nodes','no_of_edges','average_degree','average_clustering_coefficient','diameter','radius','powerlaw_exponent']]
#df_synthetic.to_csv('scripts/data/Network_Metrics_synthetic_dataset2.csv')
df_synthetic

,no_of_nodes,no_of_edges,average_degree,average_clustering_coefficient,diameter,radius,powerlaw_exponent
0,26.0,81.0,6.238131,0.703321,3,2,2.407586
1,29.0,76.0,5.260353,0.633320,4,2,2.984418
2,32.0,85.0,5.349207,0.683539,3,2,2.365894
3,34.0,111.0,6.456137,0.732987,4,2,2.362633
4,36.0,156.0,8.758000,0.713908,7,4,3.267557
...,...,...,...,...,...,...,...
95,376.0,6951.0,36.987107,0.766860,5,3,2.778269
96,380.0,5278.0,27.781883,0.768408,4,2,2.982870
97,397.0,5258.0,26.504759,0.773142,4,2,2.143431
98,415.0,2500.0,12.031852,0.707582,4,2,3.425802


# Synthetic Topology Generator

#### small_sized

In [ ]:
def small_sized():

  degreedist = random.choice(['HD','Norm'])
  if degreedist=='HD':
    high_hub=True
    Norm_hub=False
  else:
    high_hub=False
    Norm_hub=True

  if high_hub:

    while True:
      # random pick network to generate from synthetic dataset
      our_choice = random.choice(df2.index)
      extreme_range = True

      interval=df2.no_of_nodes[our_choice]*0.3
      low=df2.no_of_nodes[our_choice]-interval
      high=df2.no_of_nodes[our_choice]

      #get similar degree distribution
      df_degreedistribution=df[(low<=df.no_of_nodes) & (df.no_of_nodes<=high)]           

      #get similar average degree 
      index_degree=[]
      our_degree = df2.average_degree[our_choice]
      for index in df_degreedistribution.index:
        if (our_degree-2 > df_degreedistribution['average_degree'][index]) | ( df_degreedistribution['average_degree'][index] > our_degree+2):
          index_degree.append(index)
      df_degreedistribution = df_degreedistribution.drop(index_degree)

      #filter based on high difference of max degree threshold
      index_degree2=[]
      for index in df_degreedistribution.index:
        if (df_degreedistribution.no_of_nodes[index]-max(df_degreedistribution.degree_of_nodes[index]))/df_degreedistribution.no_of_nodes[index]>0.2:
          index_degree2.append(index)
      if len(index_degree2)>0:
        break

    all_degrees=[]
    for indexx in index_degree2:
      for degreee in df_degreedistribution.degree_of_nodes[indexx]:
        all_degrees.append(degreee)

    #define measurement based networks
    change_range = True
    edge_corners=df2.no_of_edges[our_choice]*2
    syn_nodes=int(df2.no_of_nodes[our_choice])
    cluster_coef = round(df2.average_clustering_coefficient[our_choice],2)
    dia = df2.diameter[our_choice]

    threshold=0
    while True:
      threshold+=1
      if threshold>=100000:
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      #constraint on average degree
      if edge_corners-10<=sum(nodes_degree_list)<=edge_corners+10:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          #generate havel hakimi network from sampled degree distribution
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>5:
            continue
          else:
            #reducing components
            for f in range(len(components)-1):
              for target in list(components[f+1]):
                source = random.choice(list(components[0])[:5])
                g.add_edge(source, target)     

            G=[g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)][0]
            #constraint on average clustering coefficient
            if cluster_coef-0.01<=round((nx.average_clustering(G)),2)<=cluster_coef+0.01:
              if nx.diameter(G)<=dia:
                change_range=False
                extreme_range=False
                break
              else:
                continue 
            else:
              continue
        except:
          continue

    if extreme_range:
      
      interval=df2.no_of_nodes[our_choice]*0.3
      low=df2.no_of_nodes[our_choice]-interval
      high=df2.no_of_nodes[our_choice]

      #get similar degree distribution
      df_degreedistribution=df[(low<=df.no_of_nodes) & (df.no_of_nodes<=high)]           

      #get similar average degree 
      index_degree=[]
      our_degree = df2.average_degree[our_choice]
      for index in df_degreedistribution.index:
        if (our_degree-2 > df_degreedistribution['average_degree'][index]) | ( df_degreedistribution['average_degree'][index] > our_degree+2):
          index_degree.append(index)
      df_degreedistribution = df_degreedistribution.drop(index_degree)

      #filter based on high difference of max degree threshold
      index_degree2=[]
      for index in df_degreedistribution.index:
        if (df_degreedistribution.no_of_nodes[index]-max(df_degreedistribution.degree_of_nodes[index]))/df_degreedistribution.no_of_nodes[index]>0.1:
          index_degree2.append(index)

      all_degrees=[]
      for indexx in index_degree2:
        for degreee in df_degreedistribution.degree_of_nodes[indexx]:
          all_degrees.append(degreee)

      #define measurement based networks
      edge_corners=df2.no_of_edges[our_choice]*2
      syn_nodes=int(df2.no_of_nodes[our_choice])
      cluster_coef = round(df2.average_clustering_coefficient[our_choice],2)
      dia = df2.diameter[our_choice]

      threshold=0
      while extreme_range:
        threshold+=1
        if threshold>=100000:
          Norm_hub=True
          break
        nodes_degree_list=choices(all_degrees,k=syn_nodes)
        #constraint on average degree
        if edge_corners-10<=sum(nodes_degree_list)<=edge_corners+10:
          try:
            sort_degree_list=sorted(nodes_degree_list,reverse=True) 
            #generate havel hakimi network from sampled degree distribution
            g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
            components = dict(enumerate(nx.connected_components(g)))
            if len(components)>8:
              continue
            else:
              #reducing components
              for f in range(len(components)-1):
                for target in list(components[f+1]):
                  source = random.choice(list(components[0])[:3])
                  g.add_edge(source, target)     

              G=[g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)][0]
              #constraint on average clustering coefficient
              if cluster_coef-0.1<=round((nx.average_clustering(G)),2)<=cluster_coef+0.1:
                if nx.diameter(G)<=dia:
                  change_range=False
                  extreme_range=False
                  break
                else:
                  continue 
              else:
                continue
          except:
            continue      
  
  #if extreme_range:
  #  Norm_hub=True 
  if Norm_hub:
 
    while True:
      # random pick network to generate from synthetic dataset
      our_choice = random.choice(df2.index)
      extreme_range = True

      interval=df2.no_of_nodes[our_choice]*0.3
      low=df2.no_of_nodes[our_choice]-interval
      high=df2.no_of_nodes[our_choice]

      #get similar degree distribution
      df_degreedistribution=df[(low<=df.no_of_nodes) & (df.no_of_nodes<=high)]           

      #get similar average degree 
      index_degree=[]
      our_degree = df2.average_degree[our_choice]
      for index in df_degreedistribution.index:
        if (our_degree-2 > df_degreedistribution['average_degree'][index]) | ( df_degreedistribution['average_degree'][index] > our_degree+2):
          index_degree.append(index)
      df_degreedistribution = df_degreedistribution.drop(index_degree)

      if len(df_degreedistribution.iloc[:,:4])>0:
        break

    all_degrees=[]
    for indexx in df_degreedistribution.index:
      for degreee in df_degreedistribution.degree_of_nodes[indexx]:
        all_degrees.append(degreee)

    #define measurement based networks
    change_range = True
    edge_corners=df2.no_of_edges[our_choice]*2
    syn_nodes=int(df2.no_of_nodes[our_choice])
    cluster_coef = round(df2.average_clustering_coefficient[our_choice],2)
    dia = df2.diameter[our_choice]

    threshold=0
    while True:
      threshold+=1
      if threshold>=10000:
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      #constraint on average degree
      if edge_corners-10<=sum(nodes_degree_list)<=edge_corners+10:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          #generate havel hakimi network from sampled degree distribution
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>1:
            continue
          else:
            G=[g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)][0]
            #constraint on average clustering coefficient
            if cluster_coef-0.01<=round((nx.average_clustering(G)),2)<=cluster_coef+0.01:
              if (dia-1)<=nx.diameter(G)<=dia:
                change_range=False
                extreme_range=False
                break
              else:
                continue 
            else:
              continue
        except:
          continue

    threshold=0
    while change_range:
      threshold+=1
      if threshold>=10000:
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      if edge_corners-30<=sum(nodes_degree_list)<=edge_corners+30:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True)
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>1:
            continue
          else:
            G=[g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)][0]
            if cluster_coef-0.02<=round((nx.average_clustering(G)),2)<=cluster_coef+0.02:
              if (dia-1)<=nx.diameter(G)<=dia:
                change_range=False
                extreme_range=False
                break
              else:
                continue
            else:
              continue
        except:
          continue

    if extreme_range:
      interval=df2.no_of_nodes[our_choice]*0.5
      low=df2.no_of_nodes[our_choice]-interval
      high=df2.no_of_nodes[our_choice]

      df_degreedistribution=df[(low<=df.no_of_nodes) & (df.no_of_nodes<=high)]          

      all_degrees=[]
      for indexx in df_degreedistribution.index:
        for degreee in df_degreedistribution.degree_of_nodes[indexx]:
          all_degrees.append(degreee)

    while extreme_range:
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      if edge_corners-40<=sum(nodes_degree_list)<=edge_corners+40:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>10:
            continue
          else:
            #reducing components here
            for f in range(len(components)-1):   
              source = list(components[0])[0]
              target = random.choice(list(components[f+1]))
              g.add_edge(source, target)
            G=[g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)][0]
            if cluster_coef-0.3<=round((nx.average_clustering(G)),2)<=cluster_coef+0.3:
              if nx.diameter(G)<=dia:
                change_range=False
                extreme_range=False
                break
              else:
                continue
            else:
              continue
        except:
          continue

  return G,df_degreedistribution,our_choice

#### medium_sized

In [ ]:
def medium_sized():

  degreedist = random.choice(['HD','Norm'])
  if degreedist=='HD':
    high_hub=True
    Norm_hub=False
  else:
    high_hub=False
    Norm_hub=True

  if high_hub:

    while True:
      # random pick network to generate from synthetic dataset
      our_choice = random.choice(df2.index)
      extreme_range = True

      interval=df2.no_of_nodes[our_choice]*0.3
      low=df2.no_of_nodes[our_choice]-interval
      high=df2.no_of_nodes[our_choice]

      #get similar degree distribution
      df_degreedistribution=df[(low<=df.no_of_nodes) & (df.no_of_nodes<=high)]           

      #get similar average degree 
      index_degree=[]
      our_degree = df2.average_degree[our_choice]
      for index in df_degreedistribution.index:
        if (our_degree-2 > df_degreedistribution['average_degree'][index]) | ( df_degreedistribution['average_degree'][index] > our_degree+2):
          index_degree.append(index)
      df_degreedistribution = df_degreedistribution.drop(index_degree)

      #filter based on high difference of max degree threshold
      index_degree2=[]
      for index in df_degreedistribution.index:
        if (df_degreedistribution.no_of_nodes[index]-max(df_degreedistribution.degree_of_nodes[index]))/df_degreedistribution.no_of_nodes[index]>0.2:
          index_degree2.append(index)
      if len(index_degree2)>0:
        break

    all_degrees=[]
    for indexx in index_degree2:
      for degreee in df_degreedistribution.degree_of_nodes[indexx]:
        all_degrees.append(degreee)

    #define measurement based networks
    change_range = True
    edge_corners=df2.no_of_edges[our_choice]*2
    syn_nodes=int(df2.no_of_nodes[our_choice])
    cluster_coef = round(df2.average_clustering_coefficient[our_choice],2)
    dia = df2.diameter[our_choice]

    threshold=0
    while True:
      threshold+=1
      if threshold>=100000:
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      #constraint on average degree
      if edge_corners-20<=sum(nodes_degree_list)<=edge_corners+20:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          #generate havel hakimi network from sampled degree distribution
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>10:
            continue
          else:
            #reducing components
            for f in range(len(components)-1):
              for target in list(components[f+1]):
                source = random.choice(list(components[0])[:5])
                g.add_edge(source, target)     

            G=[g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)][0]
            #constraint on average clustering coefficient
            if cluster_coef-0.01<=round((nx.average_clustering(G)),2)<=cluster_coef+0.01:
              if nx.diameter(G)<=dia:
                change_range=False
                extreme_range=False
                break
              else:
                continue 
            else:
              continue
        except:
          continue

    threshold=0
    while change_range:
      threshold+=1
      if threshold>=10000:
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      #constraint on average degree
      if edge_corners-20<=sum(nodes_degree_list)<=edge_corners+20:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          #generate havel hakimi network from sampled degree distribution
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>20:
            continue
          else:
            #reducing components
            for f in range(len(components)-1):
              for target in list(components[f+1]):
                source = random.choice(list(components[0])[:5])
                g.add_edge(source, target)     

            G=[g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)][0]
            #constraint on average clustering coefficient
            if cluster_coef-0.1<=round((nx.average_clustering(G)),2)<=cluster_coef+0.1:
              if nx.diameter(G)<=dia:
                change_range=False
                extreme_range=False
                break
              else:
                continue 
            else:
              continue
        except:
          continue

    threshold=0
    while extreme_range:
      threshold+=1
      if threshold>=100000:
        Norm_hub=True
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      #constraint on average degree
      if edge_corners-20<=sum(nodes_degree_list)<=edge_corners+20:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          #generate havel hakimi network from sampled degree distribution
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>20:
            continue
          else:
            #reducing components
            for f in range(len(components)-1):
              for target in list(components[f+1]):
                source = random.choice(list(components[0])[:5])
                g.add_edge(source, target)     

            G=[g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)][0]
            #constraint on average clustering coefficient
            if cluster_coef-0.2<=round((nx.average_clustering(G)),2)<=cluster_coef+0.2:
              if nx.diameter(G)<=dia+1:
                change_range=False
                extreme_range=False
                break
              else:
                continue 
            else:
              continue
        except:
          continue

  if Norm_hub:
    while True:
      # random pick network to generate from synthetic dataset  
      our_choice = random.choice(df2.index)
      extreme_range = True

      interval=df2.no_of_nodes[our_choice]*0.3
      low=df2.no_of_nodes[our_choice]-interval
      high=df2.no_of_nodes[our_choice]

      #get similar degree distribution
      df_degreedistribution=df[(low<=df.no_of_nodes) & (df.no_of_nodes<=high)]          

      #get similar average degree 
      index_degree=[]
      our_degree = df2.average_degree[our_choice]
      for index in df_degreedistribution.index:
        if (our_degree-2 > df_degreedistribution['average_degree'][index]) | ( df_degreedistribution['average_degree'][index] > our_degree+2):
          index_degree.append(index)
      df_degreedistribution = df_degreedistribution.drop(index_degree)

      if len(df_degreedistribution.iloc[:,:4])>0:
        break

    all_degrees=[]
    for indexx in df_degreedistribution.index:
      for degreee in df_degreedistribution.degree_of_nodes[indexx]:
        all_degrees.append(degreee)
  
    #define measurement based networks
    change_range = True
    edge_corners=df2.no_of_edges[our_choice]*2
    syn_nodes=int(df2.no_of_nodes[our_choice])
    cluster_coef = round(df2.average_clustering_coefficient[our_choice],2)
    dia = df2.diameter[our_choice]

    threshold=0
    while True:
      threshold+=1
      if threshold>=10000:
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
        
      #constraint on average degree
      if edge_corners-20<=sum(nodes_degree_list)<=edge_corners+20:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          #generate havel hakimi network from sampled degree distribution
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>10:
            continue
          else:
            #reduce components
            for f in range(len(components)-1):   
              source = list(components[0])[0]
              target = random.choice(list(components[f+1]))
              g.add_edge(source, target)

            G=g 
            #constraint on average clustering coefficient
            if cluster_coef-0.01<=round((nx.average_clustering(G)),2)<=cluster_coef+0.01:
              if (dia-1)<=nx.diameter(G)<=dia:
                change_range=False
                extreme_range=False
                break
              else:
                continue
            else:
              continue
        except:
          continue

    threshold=0
    while change_range:
      threshold+=1
      if threshold>=10000:
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      if edge_corners-30<=sum(nodes_degree_list)<=edge_corners+30:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>10:
            continue
          else:
            #reduce components
            for f in range(len(components)-1):   
              source = list(components[0])[0]
              target = random.choice(list(components[f+1]))
              g.add_edge(source, target)
            G=g 
            if cluster_coef-0.1<=round((nx.average_clustering(G)),2)<=cluster_coef+0.1:
              if nx.diameter(G)<=dia:  
                change_range=False
                extreme_range=False
                break
              else:
                continue
            else:
              continue
        except:
          continue

    if extreme_range:
      
      interval=df2.no_of_nodes[our_choice]*0.3
      low=df2.no_of_nodes[our_choice]-interval
      high=df2.no_of_nodes[our_choice]

      df_degreedistribution=df[(low<=df.no_of_nodes) & (df.no_of_nodes<=high)]           

      all_degrees=[]
      for indexx in df_degreedistribution.index:
        for degreee in df_degreedistribution.degree_of_nodes[indexx]:
          all_degrees.append(degreee)

    while extreme_range:
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      if edge_corners-50<=sum(nodes_degree_list)<=edge_corners+50:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>30:
            continue
          else:
            #reduce components
            for f in range(len(components)-1):   
              source = list(components[0])[0]
              target = random.choice(list(components[f+1]))
              g.add_edge(source, target)
            G=g 
            if cluster_coef-0.3<=round((nx.average_clustering(G)),2)<=cluster_coef+0.3:
              if nx.diameter(G)<=dia+5:   
                change_range=False
                extreme_range=False
                break
              else:
                continue
            else:
              continue
        except:
          continue

  return G,df_degreedistribution,our_choice

#### large_sized



In [ ]:
def large_sized():

  degreedist = np.random.choice(['HD','Norm'],p=[0.3,0.7])
  if degreedist=='HD':
    high_hub=True
    Norm_hub=False
  else:
    high_hub=False
    Norm_hub=True
  
  if high_hub:
    while True:
      # random pick network to generate from synthetic dataset
      our_choice = random.choice(df2.index)
      extreme_range = True

      interval=df2.no_of_nodes[our_choice]*0.3
      low=df2.no_of_nodes[our_choice]-interval
      high=df2.no_of_nodes[our_choice]

      #get similar degree distribution
      df_degreedistribution=df[(low<=df.no_of_nodes) & (df.no_of_nodes<=high)]           

      #get similar average degree 
      index_degree=[]
      our_degree = df2.average_degree[our_choice]
      for index in df_degreedistribution.index:
        if (our_degree-2 > df_degreedistribution['average_degree'][index]) | ( df_degreedistribution['average_degree'][index] > our_degree+2):
          index_degree.append(index)
      df_degreedistribution = df_degreedistribution.drop(index_degree)

      #filter based on high difference of max degree threshold
      index_degree2=[]
      for index in df_degreedistribution.index:
        if (df_degreedistribution.no_of_nodes[index]-max(df_degreedistribution.degree_of_nodes[index]))/df_degreedistribution.no_of_nodes[index]>0.2:
          index_degree2.append(index)
      if len(index_degree2)>0:
        break

    all_degrees=[]
    for indexx in index_degree2:
      for degreee in df_degreedistribution.degree_of_nodes[indexx]:
        all_degrees.append(degreee)

    #define measurement based networks
    change_range = True
    edge_corners=df2.no_of_edges[our_choice]*2
    syn_nodes=int(df2.no_of_nodes[our_choice])
    cluster_coef = round(df2.average_clustering_coefficient[our_choice],2)
    dia = df2.diameter[our_choice]

    if syn_nodes>300:
        thres1,thres2=10,1000
    else:
        thres1,thres2=10000,10000  
    threshold=0
    while True:
      threshold+=1
      if threshold>=thres1:
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      #constraint on average degree
      if edge_corners-syn_nodes<=sum(nodes_degree_list)<=edge_corners+syn_nodes:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          #generate havel hakimi network from sampled degree distribution
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>20:
            continue
          else:
            #reducing components
            for f in range(len(components)-1):
              for target in list(components[f+1]):
                source = random.choice(list(components[0])[:5])
                g.add_edge(source, target)     

            G=[g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)][0]
            #constraint on average clustering coefficient
            if cluster_coef-0.15<=round((nx.average_clustering(G)),2)<=cluster_coef+0.15:
              if nx.diameter(G)<=dia:
                change_range=False
                extreme_range=False
                break
              else:
                continue 
            else:
              continue
        except:
          continue

    threshold=0
    while change_range:
      threshold+=1
      if threshold>=thres2:
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      inter = 1.5*syn_nodes
      #constraint on average degree
      if edge_corners-inter<=sum(nodes_degree_list)<=edge_corners+inter:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          #generate havel hakimi network from sampled degree distribution
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>35:
            continue
          else:
            #reducing components
            for f in range(len(components)-1):
              for target in list(components[f+1]):
                source = random.choice(list(components[0])[:5])
                g.add_edge(source, target)     

            G=[g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)][0]
            #constraint on average clustering coefficient
            if cluster_coef-0.15<=round((nx.average_clustering(G)),2)<=cluster_coef+0.15:
              if nx.diameter(G)<=dia:
                change_range=False
                extreme_range=False
                break
              else:
                continue 
            else:
              continue
        except:
          continue

    threshold=0
    while extreme_range:
      threshold+=1
      if threshold>=10000:
        Norm_hub=True
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      inter = 1.5*syn_nodes
      #constraint on average degree
      if edge_corners-inter<=sum(nodes_degree_list)<=edge_corners+inter:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          #generate havel hakimi network from sampled degree distribution
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>50:
            continue
          else:
            #reducing components
            for f in range(len(components)-1):
              for target in list(components[f+1]):
                source = random.choice(list(components[0])[:5])
                g.add_edge(source, target)     

            G=[g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)][0]
            #constraint on average clustering coefficient
            if cluster_coef-0.2<=round((nx.average_clustering(G)),2)<=cluster_coef+0.2:
              change_range=False
              extreme_range=False
              break
            else:
              continue
        except:
          continue

  if Norm_hub:
    while True:
      # random pick network to generate from synthetic dataset
      our_choice = random.choice(df2.index)
         
      extreme_range = True

      interval=df2.no_of_nodes[our_choice]*0.3
      low=df2.no_of_nodes[our_choice]-interval
      high=df2.no_of_nodes[our_choice]

      #get similar degree distribution
      df_degreedistribution=df[(low<=df.no_of_nodes) & (df.no_of_nodes<=high)]           

      #get similar average degree 
      index_degree=[]
      our_degree = df2.average_degree[our_choice]
      for index in df_degreedistribution.index:
        if (our_degree-2 > df_degreedistribution['average_degree'][index]) | ( df_degreedistribution['average_degree'][index] > our_degree+2):
          index_degree.append(index)
      df_degreedistribution = df_degreedistribution.drop(index_degree)

      if len(df_degreedistribution.iloc[:,:4])>0:
        break

    all_degrees=[]
    for indexx in df_degreedistribution.index:
      for degreee in df_degreedistribution.degree_of_nodes[indexx]:
        all_degrees.append(degreee)

    #define measurement based networks
    change_range = True
    edge_corners=df2.no_of_edges[our_choice]*2
    syn_nodes=int(df2.no_of_nodes[our_choice])
    cluster_coef = round(df2.average_clustering_coefficient[our_choice],2)
    dia = df2.diameter[our_choice]

    if syn_nodes>340:
        thres1,thres2=1,1000
    else:
        thres1,thres2=1000,10000  
    threshold=0
    while True:
      threshold+=1
      if threshold>=thres1:
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      #constraint on average degree
      inter = syn_nodes
      if edge_corners-inter<=sum(nodes_degree_list)<=edge_corners+inter:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          #generate havel hakimi network from sampled degree distribution
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))

          if len(components)>50:
            continue
          else:
            #reduce components
            for f in range(len(components)-1):   
              source = list(components[0])[0]
              target = random.choice(list(components[f+1]))
              g.add_edge(source, target)
            G=g 
            #constraint on average clustering coefficient
            if cluster_coef-0.15<=round((nx.average_clustering(G)),2)<=cluster_coef+0.15:
              if nx.diameter(G)<=dia:
                change_range=False
                extreme_range=False
                break
              else:
                continue
            else:
              continue
        except:
          continue

    threshold=0
    while change_range:
      threshold+=1
      if threshold>=thres2:
        break
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      inter = 1.5*syn_nodes
      if edge_corners-inter<=sum(nodes_degree_list)<=edge_corners+inter:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          
          if len(components)>50:
            continue
          else:
            #reduce components
            for f in range(len(components)-1):   
              source = list(components[0])[0]
              target = random.choice(list(components[f+1]))
              g.add_edge(source, target)

            G=g 
            if cluster_coef-0.2<=round((nx.average_clustering(G)),2)<=cluster_coef+0.2:
              if nx.diameter(G)<=dia+5:
                change_range=False
                extreme_range=False
                break
              else:
                continue
            else:
              continue
        except:
          continue


    threshold=0     
    while extreme_range:
      threshold+=1
      if threshold>=10000:
        break    
      nodes_degree_list=choices(all_degrees,k=syn_nodes)
      inter = 2*syn_nodes
      if edge_corners-inter<=sum(nodes_degree_list)<=edge_corners+inter:
        try:
          sort_degree_list=sorted(nodes_degree_list,reverse=True) 
          g=nx.havel_hakimi_graph(sort_degree_list, create_using=None) 
          components = dict(enumerate(nx.connected_components(g)))
          if len(components)>50:
            continue
          else:
            #reduce components
            for f in range(len(components)-1):   
              source = list(components[0])[0]
              target = random.choice(list(components[f+1]))
              g.add_edge(source, target)       

            G=g 
            if cluster_coef-0.3<=round((nx.average_clustering(G)),2)<=cluster_coef+0.3:
              change_range=False
              extreme_range=False
              break
            else:
              continue
        except:
          continue

  return G,df_degreedistribution,our_choice


#### generate network

In [ ]:
#load real dataset
a_file = open("scripts/data/Network_Metrics_real_dataset.pkl", "rb")
data = pickle.load(a_file)
data=data.drop(columns=['repo'])
data=data.sort_values(by='no_of_nodes').reset_index(drop=True)
df=data.iloc[:,:]

# choose group from small_sized,medium_sized,large_sized
choosen_group = random.choice(['small_sized','medium_sized','large_sized'])

iterate_dic = {'small_sized':[0,16],'medium_sized':[16,43],'large_sized':[43,100]}
aa=iterate_dic[choosen_group][0]
bb=iterate_dic[choosen_group][1]

#load synthetic dataset
df2=pd.read_csv('scripts/data/Network_Metrics_synthetic_dataset.csv',index_col=0)
df2=df2.sort_values(by='no_of_nodes').reset_index(drop=True)
df2=df2.iloc[aa:bb,:].reset_index(drop=True)

#generate topology as per size
if choosen_group=='small_sized':
  G,df_degreedistribution,our_choice=small_sized()
elif choosen_group=='medium_sized':
  G,df_degreedistribution,our_choice=medium_sized()
elif choosen_group=='large_sized':
  G,df_degreedistribution,our_choice=large_sized()


In [ ]:
# cliques prioritization

In [ ]:
df_model=pd.read_csv('scripts/data/usecase_data/df_model.csv',index_col=0)

#model
X=df_model.iloc[:,[0,1,2,3,7]]
y=df_model.iloc[:,[9]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=2550)  

model = Ridge()                  
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# calculate spearman's correlation
coef, p = spearmanr(predictions, y_test)

#identify cliques in the synthetic network
from networkx import enumerate_all_cliques,find_cliques
lst_cliques=list(find_cliques(G))

#create dataframe of cliques
df_generated = pd.DataFrame([lst_cliques]).T
df_generated = df_generated.rename(columns={0:'committer'})

In [ ]:
#find predictor network centrality metrics
df_generated['degree_sum']=0.0
df_generated['betweenness_sum']=0.0
df_generated['closeness_sum']=0.0
df_generated['clustering_sum']=0.0
df_generated['eigenvector_sum']=0.0
df_generated['pagerank_sum']=0.0
df_generated['hubs_sum']=0.0
df_generated['Number of Hubs']=0

hub_limit=int(sorted(dict(nx.degree(G)).values(),reverse=True)[0]*0.5)
betweenness_dict=nx.betweenness_centrality(G)
closeness_dict=nx.closeness_centrality(G)
hubs_dict=nx.hits(G)[0]
pagerank_dict=nx.pagerank(G)
eigenvector_dict=nx.eigenvector_centrality(G)
clustering_dict=nx.clustering(G)

for index in df_generated.index:
    
    clustering_sum=0.0
    eigenvector_sum=0.0
    pagerank_sum=0.0
    hubs_sum=0.0
    count_hubs=0 
    degree_sum=0.0
    betweenness_sum=0.0
    closeness_sum=0.0
    for author in df_generated['committer'][index]:
        try:
            degree_sum+=G.degree(author)
            betweenness_sum+=betweenness_dict[author]
            closeness_sum+=closeness_dict[author] 
            clustering_sum+=clustering_dict[author]
            eigenvector_sum+=eigenvector_dict[author]
            pagerank_sum+=pagerank_dict[author]
            hubs_sum+=hubs_dict[author]
            if G.degree(author)>hub_limit:
                count_hubs+=1
        except:
            continue
            
    # take average of all centrality measures
    df_generated['degree_sum'][index]=degree_sum/len(df_generated['committer'][index])
    df_generated['betweenness_sum'][index]=betweenness_sum/len(df_generated['committer'][index])
    df_generated['closeness_sum'][index]=closeness_sum/len(df_generated['committer'][index])
    df_generated['clustering_sum'][index]=clustering_sum/len(df_generated['committer'][index])
    df_generated['eigenvector_sum'][index]=eigenvector_sum/len(df_generated['committer'][index])
    df_generated['pagerank_sum'][index]=pagerank_sum/len(df_generated['committer'][index])
    df_generated['hubs_sum'][index]=hubs_sum/len(df_generated['committer'][index])
    df_generated['Number of Hubs'][index]=count_hubs


In [ ]:
df_generated['Developers']=df_generated['committer'].apply(lambda x: len(x))

df_generated_copy=df_generated.copy()
#df_generated.to_csv('scripts/data/usecase_data/df_generated.csv')

In [ ]:
#non average centrality measures
df_generated=pd.read_csv('scripts/data/usecase_data/df_generated.csv',index_col=0)
data_lst = [df_generated]
for data in data_lst:
    data['degree_sum']=data['degree_sum']*data['Developers']
    data['betweenness_sum']=data['betweenness_sum']*data['Developers']
    data['closeness_sum']=data['closeness_sum']*data['Developers']
    data['clustering_sum']=data['clustering_sum']*data['Developers']
    data['eigenvector_sum']=data['eigenvector_sum']*data['Developers']
    data['pagerank_sum']=data['pagerank_sum']*data['Developers']
    data['hubs_sum']=data['hubs_sum']*data['Developers']

#normalize predictor metrics
cols_to_norm = ['degree_sum','betweenness_sum','closeness_sum','Developers','clustering_sum', 'eigenvector_sum', 
                'pagerank_sum','hubs_sum','Number of Hubs']
data_lst = [df_generated]
for data in data_lst:
    data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))


In [ ]:
#log transformation
df_generated=df_generated[['degree_sum','betweenness_sum','closeness_sum','Developers','hubs_sum']]
df_generated=df_generated+0.000001 

df_generated['degree_sum'] = np.log(df_generated['degree_sum'])
df_generated['Developers'] = np.log(df_generated['Developers'])
df_generated['closeness_sum'] = np.log(df_generated['closeness_sum'])
df_generated['betweenness_sum'] = np.log(df_generated['betweenness_sum'])
df_generated['hubs_sum'] = np.log(df_generated['hubs_sum'])

In [ ]:
#predict bug probability
df_generated['bug_probability']=model.predict(df_generated)

In [ ]:
# reverse log transform
df_generated.iloc[:,[0,1,2,3,4,5]] = np.exp(df_generated.iloc[:,[0,1,2,3,4,5]])
df_generated['committer']=df_generated_copy['committer']

In [ ]:
df_generated=df_generated.rename(columns={'committer':'Cliques'})
df_generated=df_generated[['Cliques','degree_sum','betweenness_sum','closeness_sum','Developers','hubs_sum','bug_probability']]
sort_cliques=df_generated.sort_values(by='bug_probability',ascending=False)
#sort_cliques.to_csv('scripts/data/usecase_data/sort_cliques.csv')
#display(HTML(sort_cliques.to_html()))

##### END